In [1]:
#@title Imports and Installations { vertical-output: true, form-width: "15%" }
!pip install pyomo
!apt install glpk-utils
#!pip install glpk
!apt-get install -y -qq glpk-utilsk

from fileinput import filename
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo as pyo
from pytest import param
import pandas as pd
import numpy as np
import os.path
import math
from datetime import datetime
from multiprocessing import Process
from google.colab import drive
import pickle
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 27 not upgraded.
Need to get 694 kB of archives.
After this operation, 1,694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libsuitesparseconfig5 amd64 1:5.7.1

In [2]:
#for google colab
#@title Optimization Model { vertical-output: true, form-width: "15%" }
class greenFuelTransportation:
    def writeDataFile(dataFileName,inputDataset):
        with open('../modelInputs/'+str(dataFileName)+'.dat', 'w') as f:
            
            
            #creating for loop for writing set structure
            setNames = ["ports","shipTypes","horizon"]
            for setName in setNames:
                f.write(f'set {setName} := ')
                for i in inputDataset[setName]:
                    f.write('%s ' % i)
                f.write(';\n\n')
     
            
            
            #simplifying writing .dat file with for loop
            paramNames = inputDataset.keys()
            
            #no param index names-for writing correct structure of .dat file
            noParamIndexNames = ["shipSpeed","lifetimeShips","discountRate"]

            singleParamIndexNames = ["capexShip","opexFixedShip","opexVariableShip","bulkSize"]

            doubleParamIndexNames = {"demand":["ports","horizon"],
                                     "length":["ports","ports"],
                                     "travelTime":["ports","ports"]}
            
            skipParamIndexNames = ["totalProduction","fuelType"]

            for paramName in paramNames:
                if paramName in setNames:
                    #skip the set values as only interested in paramNames
                    continue
                if(paramName in noParamIndexNames):
                    #single parameter value structure
                    f.write('param %s := %f; \n' % (paramName,inputDataset[paramName]))                     
                elif(paramName in singleParamIndexNames):
                    #writing correct pyomo structure for single index
                    f.write('param %s := \n' % (paramName))
                    for i in inputDataset["shipTypes"]:
                        if(i != len(inputDataset[paramName])-1):
                            f.write('%d %f \n' % (i,inputDataset[paramName][i]))
                        else:
                            f.write('%d %f' % (i,inputDataset[paramName][i]))                    
                    f.write(';\n\n')
                elif(paramName not in skipParamIndexNames):
                    #all others are double indexed-getting out specific index format for unique indices
                    #i.e. not all params have the same index format
                    indexNames = doubleParamIndexNames[paramName]
                    firstIndexName =indexNames[0]
                    secondIndexName = indexNames[1]
                    
                    lenFirstIndexName = len(inputDataset[firstIndexName])-1
                    lenSecondIndexName = len(inputDataset[secondIndexName])-1
                    
                    #writing correct pyomo structure for single index
                    f.write('param %s := \n' % (paramName))                         
                    for i in inputDataset[firstIndexName]:
                        for j in inputDataset[secondIndexName]:
                            if((i == inputDataset[firstIndexName][lenFirstIndexName]) & (j == inputDataset[secondIndexName][lenSecondIndexName])):                            
                                if((paramName == "length") or (paramName == "travelTime")):
                                    f.write('%s %s %f' % (i,j,inputDataset[paramName][i,j]))    
                                else:    
                                    f.write('%s %d %f' % (i,j,inputDataset[paramName][i][j]))    
                            else:
                                if((paramName == "length") or (paramName == "travelTime")):
                                    f.write('%s %s %f \n' % (i,j,inputDataset[paramName][i,j]))    
                                else:
                                    f.write('%s %d %f \n' % (i,j,inputDataset[paramName][i][j]))

                    f.write(';\n\n')
                    
            print("Completed data file")
            

    def main(dataFileName,inputDataset,testMode=False):
        """Daily operations of a green fuel shipping fleet

        Args:
            dataFileName (str): .dat file name which will read in/be created data for model run
            inputDataset (dict): see README for further clarification on what parameters are expected to be inputted into spreadsheet
            testMode (bool): automatically set to false, if true-will delete the .dat input file and output file associated with the dataFileName
        """ 
        
        #deleting files if test mode is activated
        if(testMode):
            try:
                os.remove(f"../modelInputs/{dataFileName}.dat")
                #os.remove(f"../modelOutputs/{dataFileName}.xlsx")
            except:
                print("")
                #print("Test mode activated but one of dat or xlsx files files already deleted")
        
            
        
        # creating optimization model with pyomo abstract representation
        model = AbstractModel()

        ################### START SETS  ###################
        
        #set of total ports that can import/export green fuel
        model.ports = Set(initialize= inputDataset["ports"])  
        model.ports.construct()                
        
                        
        #set of ship model types that can be chartered for ships in simulation
        model.shipTypes = Set(initialize= inputDataset["shipTypes"])
        
        #timesteps in simulation-based on number of months in the demand
        model.horizon = RangeSet(0,len(inputDataset["demand"][inputDataset["ports"][0]])-1)
        ################### END SETS   ###################


        ################### START PARAMETERS  ###################
        
        #Fixed OPEX for ship model m
        model.opexFixedShip = Param(model.shipTypes)
        
        #Variable OPEX (fuel costs mainly) for ship model m
        model.opexVariableShip = Param(model.shipTypes)
        
        #cargo ship green fuel capacity size for model type m 
        model.bulkSize = Param(model.shipTypes)          
        
        #demand for port p at timestep t
        model.demand = Param(model.ports,model.horizon)       
        
        #length of route from two ports (length of zero means the two ports are not connected)
        model.length = Param(model.ports,model.ports)

        #travel time of route from two ports (expressed in days
        model.travelTime = Param(model.ports,model.ports)
        
        #ship speed for simulation-assume all ships travel at the same speed
        model.shipSpeed = Param()
        
        #lifetime of ships and thus the total cost estimated out to
        model.lifetimeShips = Param()
        
        #discount rate for model
        model.discountRate = Param()

        ################### END PARAMETERS    ###################


        ################### START DECISION VARIABLES    ###################

        #flow of green fuel from port i to port j for ship s at time t
        model.X = Var(model.shipTypes,model.ports,model.ports,model.horizon,domain=NonNegativeReals,initialize=0)
        
        #whether to activate port link connection i to j for ship type model
        model.Y = Var(model.shipTypes,model.ports,model.ports,model.horizon,domain=NonNegativeIntegers,initialize=0)
        
        #fuel available at port p
        model.faPort = Var(model.ports,model.horizon,domain=NonNegativeReals,initialize=0)
        
        
        ################### END DECISION VARIABLES    ###################



        ###################     START OBJECTIVE     ###################
        #sum up the CAPEX, fixed OPEX, and variable OPEX costs for cargo ships and port storage/capacity 
        def cargoShipCosts(model):
            #sum of capex for each ship you build + the fixed costs + variable costs discounted 
            return (sum(
                        sum(
                            sum(   
                                sum(
                                    sum( 
                                        model.Y[shipType,i,j,time]*(
                                        (model.opexFixedShip[shipType] +
                                        (model.opexVariableShip[shipType]*model.travelTime[i,j])))
                                        for j in model.ports)
                                    for i in model.ports)
                            for time in model.horizon) 
                        for shipType in model.shipTypes)/(math.pow((1+inputDataset["discountRate"]),t)) 
                      for t in np.arange(inputDataset["lifetimeShips"])))

          
        def minCost_rule(model):
            return (cargoShipCosts(model))
        
        model.SystemCost = Objective(rule = minCost_rule, sense = minimize)
        
        ###################       END OBJECTIVE     ###################


        ###################       START CONSTRAINTS     ###################

        #activate port link rule implementation-can only send a ship on one port route per month
        def activatePortFlowRule(model,shipType,i,j,time):
            return(model.X[shipType,i,j,time] <= model.length[i,j]*model.Y[shipType,i,j,time]*model.bulkSize[shipType])
        model.activatePortFlowConstraint = Constraint(model.shipTypes,model.ports,model.ports,model.horizon,rule=activatePortFlowRule)        
              
        #max ship flow limited by bulk size of container ship
        def maxShipFlowRule(model,shipType,i,j,time):
            return(model.X[shipType,i,j,time] <= model.Y[shipType,i,j,time]*model.bulkSize[shipType])
        model.maxShipFlowConstraint = Constraint(model.shipTypes,model.ports,model.ports,model.horizon,rule=maxShipFlowRule)        
              

        #max ship fuel that can be sent elsewhere must be less than fuel in storage
        def maxShipFuelFlowRule(model,i,time):
            return (sum(
                sum(
                    model.X[shipType,i,j,time] for j in model.ports)
             for shipType in model.shipTypes)  <= model.faPort[i,time])
        model.maxShipFuelFlowConstraint= Constraint(model.ports,model.horizon,rule=maxShipFuelFlowRule)        
           


        #fuel availability at ports definition
        #any flows in and out storage to meet demand and then any flows in and out shipping wise
        def availFuelDefinitionRule(model,port,time):
            if(time == 0):
                return(model.faPort[port,0] ==  0)
            else:
                return(
                    (model.faPort[port,time-1] - model.demand[port,time-1] +
                     sum(
                         sum(model.X[shipType,i,port,time-1] - model.X[shipType,port,i,time-1]
                            for i in model.ports) 
                         for shipType in model.shipTypes)
                     ) == model.faPort[port,time] 
                )
        model.availFuelDefinitionConstraint = Constraint(model.ports,model.horizon,rule=availFuelDefinitionRule) 
 

        #make sure you have enough fuel available in storage for demand
        def meetDemandRule(model,port,time):
            return(model.demand[port,time] <= model.faPort[port,time])
        model.meetDemandConstraint = Constraint(model.ports,model.horizon,rule=meetDemandRule)        
    
        
        ###################       END CONSTRAINTS     ###################



        ###################          WRITING DATA       ###################
        if(os.path.isfile(f"../dataInputs/{dataFileName}.dat")):
            print(f"Data file {dataFileName} already exists!\nSkipping creating .dat file")
        else:
            #print(f"Data file {dataFileName} does not exist.\nCreating .dat file")
            greenFuelTransportation.writeDataFile(dataFileName,inputDataset)
        
        # load in data for the system
        data = DataPortal()
        data.load(filename=f"../modelInputs/{dataFileName}.dat", model=model)
        instance = model.create_instance(data)



        #instance.nodePortConnectionConstraint.pprint()    
        
        solver = SolverFactory('glpk')
        results = solver.solve(instance)
        
        
          


        #getting final objective cost
        finalSimCost = (sum(
                        sum(
                            sum(   
                                sum(
                                    sum( 
                                        instance.Y[shipType,i,j,time].value*(
                                        (instance.opexFixedShip[shipType] +
                                        (instance.opexVariableShip[shipType]*instance.travelTime[i,j])))
                                        for j in instance.ports)
                                    for i in instance.ports)
                            for time in instance.horizon) 
                        for shipType in instance.shipTypes)/(math.pow((1+inputDataset["discountRate"]),t)) 
                      for t in np.arange(inputDataset["lifetimeShips"])))


        #scaling up objective cost to DLCOA (need to factor in that run time might be less than 1 year)
        finalCost = finalSimCost*(12/(len(inputDataset["horizon"])-2))/inputDataset["totalProduction"] 

        if(results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
            # Do something when the solution in optimal and feasible
            if(inputDataset["fuelType"] == "hydrogen"):
              print(f"LCOH addition ($/kg): {finalCost}")
            elif(inputDataset["fuelType"] == "ammonia"):
              print(f"LCOA addition ($/tonne): {finalCost*1000}")              
        elif(results.solver.termination_condition == TerminationCondition.infeasible):
            # Do something when model in infeasible
            print("Problem infeasible")
        else:
            # Something else is wrong
            print(f"Solver status: {results.solver.status}")


        
            

        '''
        fuelFlowDataset = np.zeros((len(inputDataset["ships"]),len(inputDataset["horizon"])))
        for ship in np.arange(len(inputDataset["ships"])):
            for time in np.arange(len(inputDataset["horizon"])):
                totalCount = 0
                for nodeI in instance.ports:
                    for nodeJ in instance.ports:
                        if(instance.X[ship,nodeI,nodeJ,time].value == None):
                            continue
                        totalCount += instance.X[ship,nodeI,nodeJ,time].value
                fuelFlowDataset[ship][time] = totalCount
        #print("fuel flow ship")
        #print(fuelFlowDataset)
        '''
        
        
        
        
        #saving data
        masterDataFrame = {}
        
        
        
        masterDataFrame["Input Parameters"] = inputDataset
        
        #masterDataFrame["Ship Chartered Dataset"] = shipCharteredDataset

        masterDataFrame["Final Cost"] = finalCost
      
  
        
        #ship selection choice
        shipTypeDataset = np.zeros((len(inputDataset["shipTypes"]),len(inputDataset["horizon"])))
        
        for shipType in np.arange(len(inputDataset["shipTypes"])):
            for time in np.arange(len(inputDataset["horizon"])):
              shipTypeDataset[shipType][time] = sum(sum(instance.Y[shipType,i,j,time].value for j in inputDataset["ports"]) for i in inputDataset["ports"])
        masterDataFrame["Ship Type Selection"] = shipTypeDataset

        
         #looking at fuel flows for each ship type
        fuelFlowShipDataset = {}#np.zeros((len(inputDataset["ships"]),len(inputDataset["shipNodes"]),len(inputDataset["shipNodes"]),len(inputDataset["horizon"])))
        for shipType in inputDataset["shipTypes"]:
            for time in np.arange(len(inputDataset["horizon"])):
                for nodeI in instance.ports:
                    for nodeJ in instance.ports:
                        fuelFlowShipDataset[shipType,nodeI,nodeJ,time] = instance.X[shipType,nodeI,nodeJ,time].value
        masterDataFrame["Fuel Flow Ship"] = fuelFlowShipDataset

    


        #looking at fuel available in storage
        portAvailDataset = {}#np.zeros((len(inputDataset["portNodes"]),len(inputDataset["horizon"])))
        for port in inputDataset["ports"]:
            for time in np.arange(len(inputDataset["horizon"])):
                portAvailDataset[port,time] = instance.faPort[port,time].value
        masterDataFrame["Port Fuel Avail"] = portAvailDataset

        
        
        
        #structure of numpy dataframe
        #0: ship build type dataset
        #1: fuel availability on ship dataset
        #2: fuel flow dataset between nodes
        #3: ship location dataset
        #4: fuel availability on port
        #5: port capacity for storage
        #6: port transfer capacity (import + export total handling)
        
        #saving data into pickle dataframe
        output = open(f'../modelOutputs/{dataFileName}.pkl', 'wb')
        pickle.dump(masterDataFrame, output)
        output.close()   
        print(f"Output written to {dataFileName} in model outputs folder")


        #print("done")

In [ ]:
#@title Single Run model {form-width: "15%" }

os.chdir('/content/drive/MyDrive/KAUST!!/researchProj/transportationModel/src')
############ START INPUT SETTINGS ############


siteLoc = "northWest"
fuelType = "ammonia"

#avail options: Europe, Asia, EvenSplit
demandScenario = "EvenSplit"
year = "2017"

#Jan is 1, December is 12
monthStart = 1
monthEnd = 12
############ END INPUT SETTINGS ############


#dont include the .dat ending or path location
datFileName = f"{fuelType}_{siteLoc}_{demandScenario}_{year}_{monthStart}_{monthEnd}"
testMode = True

siteLocDict = {
    "northWest":"DUBA",
    "centralWest":"JEDDAH",
    "northEast":"DAMMAM"
}

inputDataset = {}
#just for initial testing input data

#sets
inputDataset["ports"] = [siteLocDict[siteLoc],"HAMBURG","TOKYO"]

#small medium large very-large
inputDataset["shipTypes"] = [0,1,2,3]



#reading in demand data and setting up monthly production list
regionDemandRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name=f'regionDemand{demandScenario}')
monthlyProduction = []


#creating subdivision for summing up monthly production
hoursInMonthList = []
for month in range(1,13):
  p = pd.Period(f'{year}-{month}-1')
  hoursInMonthList.append(p.days_in_month*24)


#reading in scale up dataset for production-different structure for hydrogen or ammonia
if(fuelType == "hydrogen"):
  productionDemandRaw = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}_base_{year}.xlsx",sheet_name="hourlyEyValueDvs")

  #calculating hours in each month and setting up first entry
  previousHour = hoursInMonthList[monthStart-1]
  monthlyProduction.append(productionDemandRaw[1][0:hoursInMonthList[monthStart-1]].sum())

  for endMonth in range(monthStart,monthEnd):
    startHour = previousHour
    endHour = previousHour + hoursInMonthList[endMonth]

    monthlyProduction.append(productionDemandRaw[1][startHour:endHour].sum())

    previousHour += hoursInMonthList[endMonth]

else:
  #calculating hours in each month but adding two datasets together
  productionDemandRaw1 = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}/{siteLoc}_part0_{year}.xlsx",sheet_name="hourlyEyValueDvs")
  productionDemandRaw2 = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}/{siteLoc}_part1_{year}.xlsx",sheet_name="hourlyEyValueDvs")

  #calculating hours in each month and setting up first entry
  previousHour = int(hoursInMonthList[0]/2)
  monthlyProduction.append(productionDemandRaw1[1][0:int((hoursInMonthList[0])/2)].sum()+productionDemandRaw2[1][0:int((hoursInMonthList[0])/2)].sum())

  for endMonth in range(monthStart,monthEnd):
    startHour = previousHour
    endHour = previousHour + int(hoursInMonthList[endMonth]/2)

    monthlyProduction.append((productionDemandRaw1[1][startHour:endHour].sum()+productionDemandRaw2[1][startHour:endHour].sum())/(0.18))

    previousHour += int(hoursInMonthList[endMonth]/2)




#going through and putting in region demand dict and getting total sim production
portDemandDict = {}
totalSimProduction = 0

for port in inputDataset["ports"]:
  if(port in ["DUBA","DAMMAM","JEDDAH"]):
    tempArray1 = -1*np.array(monthlyProduction)
    tempArray2 = np.insert(tempArray1,monthEnd-monthStart+1,0)
    portDemandDict[port] = np.insert(tempArray2,monthEnd-monthStart+1,0)
    totalSimProduction = -1*np.sum(portDemandDict[port])
  elif(port == "HAMBURG"):
    tempArray1 = np.array(monthlyProduction)*regionDemandRaw["Region 2"][(monthStart-1):(monthEnd)].to_numpy()
    tempArray2 = np.insert(tempArray1,0,0)
    portDemandDict[port] = np.insert(tempArray2,0,0)
  else:
    tempArray1 = np.array(monthlyProduction)*regionDemandRaw["Region 3"][(monthStart-1):(monthEnd)].to_numpy()
    tempArray2 = np.insert(tempArray1,0,0)
    portDemandDict[port] = np.insert(tempArray2,0,0)


#putting demand into port demand dict
inputDataset["demand"] = portDemandDict


#finsihing up horizon set
inputDataset["horizon"] = np.arange(0,len(portDemandDict[port]))


shipSettingsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='shipSettings')

systemSettingsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name = "systemSettings")

paramNames = systemSettingsRaw["ParamName"]
paramValues = systemSettingsRaw["Value"]


#adding single system param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue



shipParamNames = ["charterCost (USD/hired day)","berthingFee","fuelCost","bulkSize"]




#assuming total of four birthday days across simulation
inputDataset["opexFixedShip"] = (shipSettingsRaw["berthingFee"].to_numpy())*4


#need to convert fuel Cost to day format (from $\km to $\day using assumption of 384 km/day)
inputDataset["opexVariableShip"] = shipSettingsRaw["charterCost (USD/hired day)"].to_numpy() + (shipSettingsRaw["fuelCost"].to_numpy())

inputDataset["bulkSize"] = shipSettingsRaw["bulkSize"].to_numpy()*.94



#creating dict dataset for network connection and port lengths
portLengthsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='portLengths')
travelDurationRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='travelDuration')

inputDataset["length"] = {}
inputDataset["travelTime"] = {}

totalNodes = inputDataset["ports"]    

#no self connecting nodes and intialize all node connections to zero
for node1 in totalNodes:
  for node2 in totalNodes:
    inputDataset["length"][node1,node2] = 0
    inputDataset["travelTime"][node1,node2] = 0

#assigning correct length nodes
for (port1,port2,length,day) in zip(portLengthsRaw["Port1"],portLengthsRaw["Port2"],portLengthsRaw["Length (km)"],travelDurationRaw["Days"]):
  inputDataset["length"][port1,port2] = length
  inputDataset["travelTime"][port1,port2] = day



#calculating total production quantity
inputDataset["totalProduction"] = sum(totalSimProduction*(12/(monthEnd-monthStart+1))/(math.pow((1+inputDataset["discountRate"]),t)) for t in np.arange(inputDataset["lifetimeShips"]))



#putting in fuel type
inputDataset["fuelType"] = fuelType

print("###################")
print(f"Site Location: {siteLoc}")
print(f"Fuel Type: {fuelType}")
print(f"Demand scenario: {demandScenario}")
print(f"Year: {year}")
print(f"Start to end months: {monthStart}-{monthEnd}")
print("###################")


# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Run started:", start_time)

greenFuelTransportation.main(datFileName,inputDataset,testMode)


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))

In [3]:
#@title Batch Run Set Up {form-width: "15%" }
def runGreenFuelTransportation(siteLoc,fuelType,demandScenario,year,monthStart,monthEnd):
  #dont include the .dat ending or path location
  datFileName = f"{fuelType}_{siteLoc}_{demandScenario}_{year}_{monthStart}_{monthEnd}"
  testMode = True

  siteLocDict = {
      "northWest":"DUBA",
      "centralWest":"JEDDAH",
      "northEast":"DAMMAM"
  }

  inputDataset = {}
  #just for initial testing input data

  #sets
  inputDataset["ports"] = [siteLocDict[siteLoc],"HAMBURG","TOKYO"]

  #small medium large very-large
  inputDataset["shipTypes"] = [0,1,2,3]



  #reading in demand data and setting up monthly production list
  regionDemandRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name=f'regionDemand{demandScenario}')
  monthlyProduction = []


  #creating subdivision for summing up monthly production
  hoursInMonthList = []
  for month in range(1,13):
    p = pd.Period(f'{year}-{month}-1')
    hoursInMonthList.append(p.days_in_month*24)


  #reading in scale up dataset for production-different structure for hydrogen or ammonia
  if(fuelType == "hydrogen"):
    productionDemandRaw = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}_base_{year}.xlsx",sheet_name="hourlyEyValueDvs")

    #calculating hours in each month and setting up first entry
    previousHour = hoursInMonthList[monthStart-1]
    monthlyProduction.append(productionDemandRaw[1][0:hoursInMonthList[monthStart-1]].sum())

    for endMonth in range(monthStart,monthEnd):
      startHour = previousHour
      endHour = previousHour + hoursInMonthList[endMonth]

      monthlyProduction.append(productionDemandRaw[1][startHour:endHour].sum())

      previousHour += hoursInMonthList[endMonth]

  else:
    #calculating hours in each month but adding two datasets together
    productionDemandRaw1 = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}/{siteLoc}_part0_{year}.xlsx",sheet_name="hourlyEyValueDvs")
    productionDemandRaw2 = pd.read_excel(f"../../{fuelType}Production/modelOutputs/baseResults/{siteLoc}/{siteLoc}_part1_{year}.xlsx",sheet_name="hourlyEyValueDvs")

    #calculating hours in each month and setting up first entry
    previousHour = int(hoursInMonthList[0]/2)
    monthlyProduction.append(productionDemandRaw1[1][0:int((hoursInMonthList[0])/2)].sum()+productionDemandRaw2[1][0:int((hoursInMonthList[0])/2)].sum())

    for endMonth in range(monthStart,monthEnd):
      startHour = previousHour
      endHour = previousHour + int(hoursInMonthList[endMonth]/2)

      monthlyProduction.append((productionDemandRaw1[1][startHour:endHour].sum()+productionDemandRaw2[1][startHour:endHour].sum())/(0.18))

      previousHour += int(hoursInMonthList[endMonth]/2)




  #going through and putting in region demand dict and getting total sim production
  portDemandDict = {}
  totalSimProduction = 0

  for port in inputDataset["ports"]:
    if(port in ["DUBA","DAMMAM","JEDDAH"]):
      tempArray1 = -1*np.array(monthlyProduction)
      tempArray2 = np.insert(tempArray1,monthEnd-monthStart+1,0)
      portDemandDict[port] = np.insert(tempArray2,monthEnd-monthStart+1,0)
      totalSimProduction = -1*np.sum(portDemandDict[port])
    elif(port == "HAMBURG"):
      tempArray1 = np.array(monthlyProduction)*regionDemandRaw["Region 2"][(monthStart-1):(monthEnd)].to_numpy()
      tempArray2 = np.insert(tempArray1,0,0)
      portDemandDict[port] = np.insert(tempArray2,0,0)
    else:
      tempArray1 = np.array(monthlyProduction)*regionDemandRaw["Region 3"][(monthStart-1):(monthEnd)].to_numpy()
      tempArray2 = np.insert(tempArray1,0,0)
      portDemandDict[port] = np.insert(tempArray2,0,0)



  #putting demand into port demand dict
  inputDataset["demand"] = portDemandDict


  #finsihing up horizon set
  inputDataset["horizon"] = np.arange(0,len(portDemandDict[port]))


  shipSettingsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='shipSettings')

  systemSettingsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name = "systemSettings")

  paramNames = systemSettingsRaw["ParamName"]
  paramValues = systemSettingsRaw["Value"]


  #adding single system param value data
  for paramName,paramValue in zip(paramNames,paramValues):
      inputDataset[paramName] = paramValue



  shipParamNames = ["charterCost (USD/hired day)","berthingFee","fuelCost","bulkSize"]




  #assuming total of four birthday days across simulation
  inputDataset["opexFixedShip"] = (shipSettingsRaw["berthingFee"].to_numpy())*4


  #need to convert fuel Cost to day format (from $\km to $\day using assumption of 384 km/day)
  inputDataset["opexVariableShip"] = shipSettingsRaw["charterCost (USD/hired day)"].to_numpy() + (shipSettingsRaw["fuelCost"].to_numpy())

  inputDataset["bulkSize"] = shipSettingsRaw["bulkSize"].to_numpy()*.94



  #creating dict dataset for network connection and port lengths
  portLengthsRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='portLengths')
  travelDurationRaw = pd.read_excel(f"../dataInputs/{fuelType}InputSheet.xlsx",sheet_name='travelDuration')

  inputDataset["length"] = {}
  inputDataset["travelTime"] = {}

  totalNodes = inputDataset["ports"]    

  #no self connecting nodes and intialize all node connections to zero
  for node1 in totalNodes:
    for node2 in totalNodes:
      inputDataset["length"][node1,node2] = 0
      inputDataset["travelTime"][node1,node2] = 0

  #assigning correct length nodes
  for (port1,port2,length,day) in zip(portLengthsRaw["Port1"],portLengthsRaw["Port2"],portLengthsRaw["Length (km)"],travelDurationRaw["Days"]):
    inputDataset["length"][port1,port2] = length
    inputDataset["travelTime"][port1,port2] = day



  #calculating total production quantity
  inputDataset["totalProduction"] = sum(totalSimProduction*(12/(monthEnd-monthStart+1))/(math.pow((1+inputDataset["discountRate"]),t)) for t in np.arange(inputDataset["lifetimeShips"]))



  #putting in fuel type
  inputDataset["fuelType"] = fuelType

  #running model
  greenFuelTransportation.main(datFileName,inputDataset,testMode)


In [ ]:
#@title Batch Run Execute {form-width: "15%" }

os.chdir('/content/drive/MyDrive/KAUST!!/researchProj/transportationModel/src')

############ START INPUT SETTINGS ############


siteLocs = ["northWest","centralWest","northEast"]
fuelTypes = ["ammonia"]

#avail options: Europe, Asia, EvenSplit
#demandScenarios = ["Asia","Europe"]
#demandScenarios = ["EvenSplit"]
demandScenarios = ["Asia","Europe","EvenSplit"]
years = ["2017","2018","2019"]


#Jan is 1, December is 12
#can do full year for hydrogen (for Asia and Europe) but only half a year for all ammonia and hydrogen even split
monthStart = 6
monthEnd = 12
############ END INPUT SETTINGS ############

print("###################")
print(f"Fuel Types: {fuelTypes}")
print(f"Demand scenarios: {demandScenarios}")
print(f"Years: {years}")
print(f"Month start-month end: {monthStart}-{monthEnd}")
print("###################")


# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Run started:", start_time)

#running through for assigning all runs
processes = []
for siteLoc in siteLocs:
  for fuelType in fuelTypes:
    for demandScenario in demandScenarios:
      for year in years:
        p = Process(target=runGreenFuelTransportation, args=(siteLoc,fuelType,
                                                             demandScenario,year,
                                                             monthStart,monthEnd))
        processes.append(p)
        p.start()

print(f"Total processes to run: {len(siteLocs)*len(fuelTypes)*len(demandScenarios)*len(years)}")

# completing process
for p in processes:
    p.join()


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))

###################
Fuel Types: ['ammonia']
Demand scenarios: ['Asia', 'Europe', 'EvenSplit']
Years: ['2017', '2018', '2019']
Month start-month end: 6-12
###################
Run started: 00:36:58
Total processes to run: 27
